In [10]:
import numpy as np
import pandas as pd
from datetime import date, datetime

In [3]:
YEAR = 1992
COLS_NEEDED = [
    "FIRE_YEAR",
    "DISCOVERY_DATE",
    "DISCOVERY_DOY",
    "DISCOVERY_TIME",
    "STAT_CAUSE_CODE",
    "STAT_CAUSE_DESCR",
    "CONT_DATE",
    "CONT_DOY",
    "CONT_TIME",
    "FIRE_SIZE",
    "FIRE_SIZE_CLASS",
    "LATITUDE",
    "LONGITUDE",
    "STATE",
    "COUNTY"
]

In [4]:
with open('./wildfires_%d.csv' % YEAR) as csvfile:
    df = pd.read_csv(csvfile)
    df = df[COLS_NEEDED]

/usr/local/Cellar/python3/3.6.0/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (10,11,12,18,35) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [15]:
def get_season(day, Y):
    seasons = [('winter', (date(Y,  1,  1),  date(Y,  3, 20))),
              ('spring', (date(Y,  3, 21),  date(Y,  6, 20))),
              ('summer', (date(Y,  6, 21),  date(Y,  9, 22))),
              ('autumn', (date(Y,  9, 23),  date(Y, 12, 20))),
              ('winter', (date(Y, 12, 21),  date(Y, 12, 31)))]
    for season in seasons:
        name = season[0]
        start, end = season[1]
        if (start.timetuple().tm_yday <= day <= end.timetuple().tm_yday):
            return name
    return None

def julian_to_datetime(jd_series):
    # https://www.kaggle.com/rtatman/188-million-us-wildfires/discussion/39627#222290
    epoch = pd.to_datetime(0, unit='s').to_julian_date()
    return pd.to_datetime(jd_series - epoch, unit='D')

def difference_in_seconds():
    start_times = df['DISCOVERY_TIME']
    end_times = df['CONT_TIME']
    return pd.to_datetime(end_times) - pd.to_datetime(start_times)

def num_to_time(num_time):
    # num_time is in 24-hour format - so 0 to 2359
    chars = list(str(int(num_time)))
    res = []
    for i in range(len(chars)-1, -1, -1):
      res.append(chars[i])
      if len(res) == 2:
        res.append(":")
    result = ''.join(res[::-1])
    if len(result) == 1: result = "00:0" + result
    if result[0] == ":": result = "00" + result
    return result

['AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'FL', 'GA', 'HI', 'ID', 'IL', 'IN', 'IA', 'KS', 'KY', 'LA', 'ME', 'MD', 'MA', 'MI', 'MN', 'MS', 'MO', 'MT', 'NE', 'NV', 'NH', 'NJ', 'NM', 'NY', 'NC', 'ND', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT', 'VT', 'VA', 'WA', 'WV', 'WI', 'WY']


In [6]:
# Clean
df = df.dropna(subset=['CONT_DATE', 'DISCOVERY_DATE', 'CONT_TIME', 'DISCOVERY_TIME'])
df['CONT_TIME'] = df.apply(lambda row: num_to_time(row.CONT_TIME), axis=1)
df['DISCOVERY_TIME'] = df.apply(lambda row: num_to_time(row.DISCOVERY_TIME), axis=1)
df['CONT_DATE'] = df.apply(lambda row: julian_to_datetime(row.CONT_DATE), axis=1)
df['DISCOVERY_DATE'] = df.apply(lambda row: julian_to_datetime(row.DISCOVERY_DATE),axis=1)
df['season'] = df.apply(lambda row: get_season(row.DISCOVERY_DOY,YEAR), axis=1)

In [7]:
full_disc_date = pd.to_datetime(df['DISCOVERY_DATE'].dt.strftime("%Y-%m-%d") + ' ' + df['DISCOVERY_TIME'])
full_cont_date = pd.to_datetime(df['CONT_DATE'].dt.strftime("%Y-%m-%d") + ' ' + df['CONT_TIME'])
df['duration'] = full_cont_date - full_disc_date

In [32]:
US_STATE_CODES = ['AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DC', 'DE', 'FL', 'GA', 'HI', 'ID', 'IL', 'IN', 'IA', 'KS', 'KY', 'LA', 'ME', 'MD', 'MA', 'MI', 'MN', 'MS', 'MO', 'MT', 'NE', 'NV', 'NH', 'NJ', 'NM', 'NY', 'NC', 'ND', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT', 'VT', 'VA', 'WA', 'WV', 'WI', 'WY']
SEASONS = ['winter','spring','summer','autumn']
STAT_CAUSE_CODES = list(range(1,14))

In [35]:
# TODO: Create classifier
# - extract features for each row
# - create labels for each row (this should just be getting the fire class size and mapping it to an integer)
# - split data into train + test sets (shuffle data first?)
# - train data using a model (need to look this up)
# - test for accuracy

label_names = [chr(ord('A') + i) for i in range(ord('G') - ord('A') + 1)]

# features: 'state', 'season', 'cause', 'duration' (maybe)

def extract_features(row):
    return [US_STATE_CODES.index(row.STATE), SEASONS.index(row.season), int(row.STAT_CAUSE_CODE)]

In [44]:
feature_df = df[['STATE', 'STAT_CAUSE_CODE', 'season']]
X = [extract_features(row) for _,row in feature_df.iterrows()]
y = df['FIRE_SIZE_CLASS'].map(lambda fire_class: ord(fire_class) - ord('A'))

# https://stackoverflow.com/a/4602224/8109239
def unison_shuffled_copies(a, b):
    assert len(a) == len(b)
    p = np.random.permutation(len(a))
    return np.asarray(a)[p], np.asarray(b)[p]

X,y = unison_shuffled_copies(X,y)

In [45]:
# Split data into training and test sets
train_size = 15000
test_size = len(X) - train_size
X_train, y_train = X[:train_size], y[:train_size]
X_test, y_test = X[train_size+1:], y[train_size+1:]

In [50]:
from sklearn.naive_bayes import GaussianNB
clf = GaussianNB()
clf.fit(X, y)
print(clf.predict([[0, 5, 1]]))

[1]
